In [1]:
import numpy as np
import pandas as pd

import torch
import random

random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed_all(42)

In [2]:
import torchvision
import torchvision.transforms as transforms
import time
import matplotlib.pyplot as plt

from torch.autograd import Variable
from torchvision import models

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [4]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
submit = pd.read_csv('sample_submit.csv')

In [5]:
train_x = train.drop(['Category'], axis = 1)
train_y = train.Category

test_x = test

In [6]:
train_x_np = train_x.to_numpy()#.astype('float')
test_x_np = test_x.to_numpy()#.astype('float')

In [7]:
train_x_np = train_x_np/255.0
test_x_np = test_x_np/255.0

In [8]:
train_x_np = train_x_np.reshape(-1, 1, 28, 28)
test_x_np = test_x_np.reshape(-1, 1, 28, 28)

In [9]:
#train_x_np = np.expand_dims(train_x_np, axis = 3)
print(np.shape(train_x_np))
train_x_np = np.repeat(train_x_np, 3, axis = 1)
print(np.shape(train_x_np))

#test_x_np = np.expand_dims(test_x_np, axis = 1)
print(np.shape(test_x_np))
test_x_np = np.repeat(test_x_np, 3, axis = 3)
print(np.shape(test_x_np))

(60000, 1, 28, 28)
(60000, 3, 28, 28)
(10000, 1, 28, 28)
(10000, 1, 28, 84)


In [10]:
print(len(train_x_np))

60000


In [11]:
#from tensorflow.keras.preprocessing.image import load_img, img_to_array, array_to_img

#def change_size(image):
#    img = array_to_img(image, scale=False) #returns PIL Image
#    img = img.resize((224, 244)) #resize image
#    img = img.convert(mode='RGB') #makes 3 channels
#    arr = img_to_array(img) #convert back to array
#    return arr.astype(np.float64)

In [12]:
#{uint8, int16, uint32, float32}-> ToPILImage 지원 데이터 형식

In [13]:
#change = transform

#for i in range(len(train_x_np)):
#    chage(train_x_np[i])
#    print(change)
#train_x_tf = change(train_x_np)
#train_x_tf = [change_size(img) for img in train_x_np]
#print(np.shape(train_x_tf))

In [14]:
train_x_tensor = torch.FloatTensor(train_x_np)
train_y_tensor = torch.LongTensor(train_y.values)
test_x_tensor = torch.FloatTensor(test_x_np)

In [15]:
train_dataset = torch.utils.data.TensorDataset(train_x_tensor, train_y_tensor)
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size =1000, shuffle = True)
test_dataset = torch.utils.data.TensorDataset(test_x_tensor)
testloader = torch.utils.data.DataLoader(test_dataset, batch_size =1000, shuffle = True)

In [16]:
resnet = models.resnet50(pretrained=True)
resnet.to(device)

print(resnet)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [17]:
#resnet.conv1 = torch.nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
resnet.fc = torch.nn.Linear(2048, 10, bias = True) #out feature 변경
#resnet.classifier[6].out_features = 10
resnet.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [18]:
#for param in resnet.parameters():
#    param.requires_grad = False
#resnet.fc.weight.requires_grad = True
#resnet.fc.bias.requires_grad = True

In [ ]:
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(resnet.parameters(), lr = 0.001, momentum=0.9)

In [20]:
def validate(model, test_dataloader):
    model.eval()
    val_running_loss = 0.0
    val_running_correct = 0
    for int, data in enumerate(test_dataloader):
        data, target = data[0].to(device), data[1].to(device)
        output = model(data)
        loss = criterion(output, target)
        
        val_running_loss += loss.item()
        _, preds = torch.max(output.data, 1)
        val_running_correct += (preds == target).sum().item()
    
    val_loss = val_running_loss/len(test_dataloader.dataset)
    val_accuracy = 100. * val_running_correct/len(test_dataloader.dataset)
    
    return val_loss, val_accuracy

In [21]:
def train(model, train_dataloader):
    model.train()
    train_running_loss = 0.0
    train_running_correct = 0
    for i, data in enumerate(train_dataloader):
        data, target = data[0].to(device), data[1].to(device)
        #data = torch.reshape(data, (-1,1,224,224))
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        train_running_loss += loss.item()
        _, preds = torch.max(output.data, 1)
        train_running_correct += (preds == target).sum().item()
        loss.backward()
        optimizer.step()
    train_loss = train_running_loss/len(train_dataloader.dataset)
    train_accuracy = 100. * train_running_correct/len(train_dataloader.dataset)
 
    print(f'Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.2f}')
    
    return train_loss, train_accuracy

In [22]:
train_loss , train_accuracy = [], []
val_loss , val_accuracy = [], []
start = time.time()
for epoch in range(50):
    train_epoch_loss, train_epoch_accuracy = train(resnet, trainloader)
    #val_epoch_loss, val_epoch_accuracy = validate(resnet, testloader)
    train_loss.append(train_epoch_loss)
    train_accuracy.append(train_epoch_accuracy)
    #val_loss.append(val_epoch_loss)
    #val_accuracy.append(val_epoch_accuracy)

    #print(train_accuracy, val_accuracy)
    
end = time.time()
 
print((end-start)/60, 'minutes')

C:\Users\yoons\anaconda3\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Train Loss: 0.0020, Train Acc: 37.11
Train Loss: 0.0015, Train Acc: 63.05
Train Loss: 0.0012, Train Acc: 67.89
Train Loss: 0.0011, Train Acc: 70.25


KeyboardInterrupt: 

In [ ]:
resnet.eval()
predict = resnet(test_x_tensor.to(device))

In [ ]:
predict = torch.argmax(predict, dim = 1)

In [ ]:
predict

In [ ]:
submit.Category = predict.detach().cpu().numpy()

In [ ]:
submit.to_csv('2021_11_03_2.csv', index= False) #Adam 10 epoch : 80.03 0.827

In [ ]:
#SGD 50epoch : 84.72